In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import numpy as np
from scipy.optimize import minimize

In [2]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/exp/"
    data_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/"
    filename = "exp00060"

In [3]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')
kun_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052.csv')
kun_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052_oof_pred.csv')
yuji_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp065.csv')
yuji_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp065.csv')
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [5]:
yuji_oof

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,...,pred_30,pred_31,pred_32,pred_33,pred_34,pred_35,pred_36,pred_37,pred_38,pred_39
0,2,2954.247573,2735.110086,3288.647709,2893.840210,3253.350282,2885.690295,3202.103770,2799.365349,2989.931642,...,2846.925805,3310.143650,3429.863903,2884.781039,3109.509196,3182.319352,2966.910201,3300.768016,2975.341780,2996.449911
1,25,7931.462351,6944.616443,7214.909240,7252.552211,7198.146932,6827.863480,7136.463291,7656.661603,7039.997672,...,7297.023342,8078.577311,8147.178352,7494.219142,7334.249575,9235.322324,7533.366722,7830.543987,7985.342379,7729.290281
2,26,5843.172706,5827.250371,6115.096360,5831.436096,5819.394501,6109.653073,5937.286683,5893.277859,6008.794932,...,5840.064314,5717.486835,5813.182519,5555.230170,6030.417248,5972.490854,4992.652219,5649.662585,6276.942074,6394.837313
3,51,2857.775866,3369.341416,3117.966141,3300.098540,3334.328270,3239.576316,3168.865643,2931.111896,3368.678163,...,3462.188915,2807.625463,3244.736553,3177.227074,3274.527857,2373.194810,3024.954490,3162.226422,3337.436558,3635.016537
4,52,8220.237821,7545.993230,8159.395741,8416.911652,7936.652895,7614.765389,8401.374941,7866.482215,8084.295455,...,9277.483426,8175.521660,8852.739742,7836.161295,8751.867572,8583.541673,8421.348014,8536.497119,8841.259839,8298.166888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27476,6982.456164,6547.126657,6272.924314,6883.816792,6689.595713,6897.962840,7287.918787,6927.718233,6960.224224,...,6992.435763,7297.258404,7336.679708,6882.452447,8037.155703,6886.160561,7427.216351,7152.854136,7135.735769,7789.714663
27528,27496,7991.618468,7182.188171,6341.634653,7276.265285,7334.970227,7505.917690,8018.629653,7993.739532,7144.644201,...,8015.778445,8189.397554,7030.542151,7433.594815,6896.194195,8456.167934,7801.142566,7952.634400,7323.871681,7586.437759
27529,27506,5841.883459,6062.253497,6325.240063,6711.844782,5872.272990,5930.771389,5433.789309,5979.625795,5746.513645,...,5448.763551,5692.521552,5658.925994,5743.348739,5152.747145,5334.061419,5210.396799,5693.684591,5029.697725,3941.918948
27530,27516,7885.338899,6749.518405,7498.278543,8136.739409,8110.431959,7111.222431,7208.258953,7391.535836,7542.271496,...,7444.799207,7930.943735,8011.671205,8297.678154,7606.756546,8721.782137,7465.368541,8222.872448,7824.281691,7927.535191


In [6]:
df

,id,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,...,pred_31,pred_32,pred_33,pred_34,pred_35,pred_36,pred_37,pred_38,pred_39,price
0,0,7118.9443,7627.1987,7084.0015,7043.0950,7122.7530,7335.2310,7350.0060,9836.8750,8781.6880,...,7502.731399,9046.083334,9790.207700,7999.621081,9179.605316,7404.460231,10863.206534,7674.566326,10945.332692,27587
1,1,3660.8042,3526.6816,3732.6855,3593.2502,3596.9211,3562.2925,3922.8992,3761.7544,3852.5068,...,3681.464095,3648.063325,4057.324098,3423.298315,4256.800800,3937.424257,4012.205191,3871.318876,4087.226759,4724
2,2,2883.7390,3149.7769,2995.7760,3038.0789,2876.8310,2982.5576,2787.5325,2911.1800,3172.8408,...,3310.143650,3429.863903,2884.781039,3109.509196,3182.319352,2966.910201,3300.768016,2975.341780,2996.449911,10931
3,3,8650.6270,8943.9030,9243.1750,8973.2610,9158.7180,8505.1610,8713.2310,8287.6740,8649.6400,...,8287.367768,8814.043198,8434.204563,8292.250116,8010.910553,8092.324790,8698.940654,8281.208121,7630.436524,16553
4,4,4031.1462,4406.5480,4107.9190,4059.3975,4229.2666,4429.8930,4209.0796,4026.7249,4228.1797,...,3900.863059,4312.082422,4518.990036,4405.510685,4745.297877,4280.133207,4148.129445,4364.547169,4530.330211,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,12879.4610,13338.9980,12310.2180,12790.3090,12681.5170,12661.3610,11953.5220,12138.8550,12707.8850,...,12110.394159,10867.413943,12265.167646,12634.088664,11254.044256,13375.023073,11904.047114,8838.212135,12670.297997,32212
27528,27528,6693.9270,6920.0894,6770.7925,6634.9660,7131.3125,6796.1294,6812.7780,8508.7300,7977.3506,...,7058.550684,6835.992975,6627.585070,7338.824115,6690.876650,6201.038189,6748.852167,8017.979192,6076.712101,5400
27529,27529,14174.7040,15265.7090,14390.8955,14423.8730,12980.9630,14560.7450,14018.7240,13443.1000,13441.5670,...,13052.062618,13502.897379,16729.110952,15889.839699,12465.315027,14256.179133,13746.209632,11557.775566,14097.741540,22227
27530,27530,7086.4136,6367.2173,6483.3950,7116.9200,7356.8930,7240.1910,7432.3013,7450.1953,7293.5034,...,6448.413794,6630.474905,6776.007422,6378.505290,8041.334627,6761.032776,7934.069009,6651.966732,6327.830358,3054


In [7]:


# あなたの予測結果の列名のプレフィックスと数
kun_prefix = "kun_pred_"
kun_num = 10

# もう一人の人の予測結果の列名のプレフィックスと数
other_prefix = "pred_"
other_num = 40

def objective(weights):
    kun_preds = [df[f"{kun_prefix}{idx}"] * weights[idx] for idx in range(kun_num)]
    other_preds = [df[f"{other_prefix}{idx}"] * weights[idx+kun_num] for idx in range(other_num)]

    df["pred"] = sum(kun_preds + other_preds)

    score = get_score(y_true=df["price"], y_pred=df["pred"])
    return score

# 初期の重みを設定（すべての予測の重みを等しくする）
initial_weights = [1 / (kun_num + other_num)] * (kun_num + other_num)

result = minimize(objective, initial_weights, method="nelder-mead")

best_weights = result.x

print("Optimized weights:", best_weights)
print("Best score with optimized weights:", result.fun)

Optimized weights: [ 0.13662669 -0.02017968  0.10669767 -0.17658163 -0.00957377 -0.05731034
 -0.08671983  0.10836322 -0.08489309 -0.05493305 -0.05651993 -0.09013278
  0.00613001 -0.15112548  0.00140576  0.19941601  0.01786774  0.12466984
  0.02560436 -0.12080012  0.23841526  0.13518739  0.25291471 -0.1063289
  0.0701534  -0.05309884 -0.03655411 -0.11306455 -0.01507759 -0.02352633
  0.1093787  -0.01051545  0.05548014  0.09774625  0.13797646 -0.0592736
 -0.05804345  0.04892098  0.1582608   0.18467935 -0.0679835   0.0406062
  0.03606147  0.04429405 -0.03004079  0.01238882 -0.03729647  0.11335512
 -0.01006969  0.05673697]
Best score with optimized weights: 43.56340118610674


In [12]:
# 最適化された重みを使用してOOFデータの予測を計算
kun_oof_preds = [df[f"{kun_prefix}{idx}"] * best_weights[idx] for idx in range(kun_num)]
other_oof_preds = [df[f"{other_prefix}{idx}"] * best_weights[idx+kun_num] for idx in range(other_num)]
df["oof_pred"] = sum(kun_oof_preds + other_oof_preds)

# 結果をCSVファイルとして保存
optimized_oof_filepath = CFG.save_dir + f"kun_{CFG.filename}_oof_pred.csv"
df[["id", "oof_pred"]].to_csv(optimized_oof_filepath, index=False)

In [14]:
pd.read_csv(optimized_oof_filepath)

,id,oof_pred
0,0,9322.824825
1,1,3891.997620
2,2,2971.556923
3,3,7959.507481
4,4,4375.457013
...,...,...
27527,27527,12719.460234
27528,27528,6836.122512
27529,27529,14140.050721
27530,27530,6563.780784


In [10]:
get_score(df['price'], pd.read_csv(optimized_oof_filepath)['optimized_pred'])

43.56340118610674

In [11]:
# 最適化された重みを使用してテストデータの予測を計算
kun_preds = [test[f"kun_pred_{idx}"] * best_weights[idx] for idx in range(10)]
other_preds = [test[f"pred_{idx}"] * best_weights[idx+kun_num] for idx in range(40)]
test["pred"] = sum(kun_preds + other_preds)

# 結果をCSVファイルとして保存
test[["id", "pred"]].to_csv(CFG.save_dir + f"kun_{CFG.filename}.csv", index=False, header=None)